In [199]:
%matplotlib qt5
# %matplotlib inline

In [200]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.optimize import curve_fit


from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.dpi'] = 150

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle=['solid', 'dashed', 'dotted', 'dashdot']
marker_cycle = ['o', '*', 's', '^']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load experiments

In [201]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

### Set save data

In [202]:
save_data = False

In [203]:
def prev_data(expt_path, filename=None):
    """Reopen saved data"""
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [204]:
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_241025')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [205]:
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Load config

In [230]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [207]:
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Fermium
print(list(im))

['Pyro.NameServer', 'Qick_QRAM']


In [208]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

## Parameter

In [209]:
qubit_i = 2

# Rabi versus amplitude 

We first want to know the relationship between the gain and rabi drive strength

In [225]:
# gain_start = 100 
# gain_stop = 1000
# gain_pts = 5
# gain_vec = np.linspace(gain_start, gain_stop, gain_pts)

reps = 1
rounds = 100 

t_start = 0.1
t_stop = 0.5 # us
t_pts = 300
t_step = (t_stop - t_start) / t_pts
print(f'Time step is {t_step} us')
t_step = soc.cycles2us(soc.us2cycles(t_step))
print(f'Time step is {t_step} cycles')

rabi_freq_vec = np.zeros(gain_pts)
rabi_freq_vec_err = np.zeros(gain_pts)

save_data = True

Time step is 0.0013333333333333335 us
Time step is 0.0023251488095238095 cycles


NameError: name 'gain_pts' is not defined

In [ ]:

RabiFreqExpt = Experiment(
                        path=expt_path,
                        prefix=f"rabi_freq_{qubit_i}",
                        config_file=config_path)

RabiFreqExpt.data = dict(amps=[], times=[], rabi_freq = [], gain_pts=[])
RabiFreqExpt.data['gain_pts'] = gain_vec

time_vec = []
amps_vec = []


for idx_g, g in enumerate(gain_vec):
    print(f"Gain {g} ({idx_g+1}/{len(gain_vec)})")

    lengthrabi = meas.LengthRabiExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"length_rabi_qubit{qubit_i}",
        config_file=config_path,
    )
    lengthrabi.cfg = AttrDict(deepcopy(yaml_cfg))
    
    lengthrabi.cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i*5] = int(g)

    lengthrabi.cfg.expt = dict(
        start=t_start, # pulse length (sigma) [us]
        step=t_step, # [us]
        expts=t_pts,
        reps=reps*rounds,
        pulse_type='gauss',
        checkEF=False, 
        qTest=qubit_i,
        qZZ=None,
    )

    import Pyro4.util
    try:
        lengthrabi.go(analyze=False, display=False, progress=True, save=False)
        
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
        

    amps = lengthrabi.data['amps']
    time = lengthrabi.data['xpts']
    # compute FFT
    amp_fft = fft(amps)
    freq = fftfreq(len(amps), time[1] - time[0])
    
    # find the peak
    idx = np.argmax(np.abs(amp_fft[1:])) + 1
    freq_peak = freq[idx]
    

    rabi_freq_vec[idx_g] = freq_peak/2

    
    if save_data:
        time_vec.append(time)
        amps_vec.append(amps)
        
        
    


In [ ]:
# fit with a linear function

def linear(x, a, b):
    return a*x + b

popt, pcov = curve_fit(linear, gain_vec, rabi_freq_vec)

a = popt[0]
a_err = np.sqrt(pcov[0,0])
b = popt[1]
b_err = np.sqrt(pcov[1,1])


if save_data:
    RabiFreqExpt.data['times'] = time_vec
    RabiFreqExpt.data['amps'] = amps_vec
    # divide by 2 to get the Rabi frequency ( given we apply two pi/2 actually)
    RabiFreqExpt.data['rabi_freq'] = rabi_freq_vec
    RabiFreqExpt.data['fit'] = [a, b]
    RabiFreqExpt.data['fit_err'] = [a_err, b_err]
    RabiFreqExpt.save_data()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(gain_vec, rabi_freq_vec, 'o-')
ax.plot(gain_vec, linear(gain_vec, *popt), '--', color='black')

text = f"Fit: {a*1000:.2f} x/1000 + {b:.2f}\n"
ax.text(0.1, 0.9, text, transform=ax.transAxes, fontsize=12, verticalalignment='top')

ax.set_xlabel('Gain')
ax.set_ylabel('Rabi frequency (MHz)')
ax.set_title(f'Rabi frequency vs Gain for qubit {qubit_i}')
fig.tight_layout()

In [339]:
filename = yaml_cfg.device.qubit.pulses.pihalf_ge_robust.filename[qubit_i]
path = os.path.join('S:\QRAM\qram_4QR2\optctrl_pulses', f"{filename}.npz")
nb_qb = 4


# open the file
scaling = 1

with np.load(path) as data:
    times = data['times']*scaling
    I = []
    Q = []
    for i in range(nb_qb):
        if 'I_'+str(i) in data:
            I.append(data['I_'+str(i)]/1e6/scaling)
            Q.append(data['Q_'+str(i)]/1e6/scaling)
        else:
            I.append(np.zeros(times.shape))
            Q.append(np.zeros(times.shape))

In [340]:
fig, ax = plt.subplots(figsize=(4,3))

for i in range(len(I)):
    ax.plot(times, I[i], label=f'I{i}')
    ax.plot(times, Q[i], label=f'Q{i}')
    
ax.set_xlabel('Time (us)')
ax.set_ylabel('Amplitude (MHz)')

ax.legend()
fig.tight_layout()
    

In [ ]:
# for all qubit take the maximum of the I and Q

gain = np.zeros(len(I), dtype=int)

for i in range(len(I)):
    amp_I = np.max(np.abs(I[i]))
    amp_Q = np.max(np.abs(Q[i]))
    amp_max = np.max([amp_I, amp_Q])
    if amp_max == 0:
        gain[i] = 0
    else:
        gain_max = (amp_max - b) / a
        # for some reason the gain we need to set is twice the value
        gain[i] = gain_max*2
        
        

# First round of error amplification to get the gain right

In [210]:
gain_scale = np.linspace(0.1, 1.5, 15)

x_tab = []
p_tab = []

qubit_i=1
gain = 5000

with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)


for s in gain_scale:

    npulsecalib = meas.NPulseExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"NPulseExptQ{qubit_i}",
        config_file=config_path,
    )

    npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
    npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(gain*s)
    # npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(1)
    # npulsecalib.cfg.device.qubit.f_ge_robust[qubit_i*5] = 3448.5530242338505

    npulsecalib.cfg.expt = dict(
        start=1, # number gate repetitions
        step=10,
        expts=1,
        reps=1000,
        loops=10,
        pulse_type='robust',
        checkEF=False, 
        qTest=qubit_i,
        qZZ=None,
        test_pi_half=True,
        error_amp=True,
        skip_first_pi2=True,
        post_process='threshold',
        singleshot_reps=5000,
        resonator_reset=[0, 2, 3],
        full_mux_expt=True
    )

    # print(npulsecalib.cfg)
    import Pyro4.util
    try:
        npulsecalib.acquire(progress=True, debug=False)
    except Exception:
        print("Pyro traceback:")
        print("".join(Pyro4.util.getPyroTraceback()))
        
        
    data = npulsecalib.data
    
    p_vec = data['avgi'][-1]
    x_vec = data['xpts'][-1]
    
    p_tab.append(p_vec)
    x_tab.append(x_vec)

  0%|          | 0/2 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
Qubit  (1)
g unrotated averages:
Amps 270.18701956064064 +/- 64.4501961888381
e unrotated averages:
Amps 363.93986966022896 +/- 67.34874860430958
ge fidelity (%): 54.91560312062411 	 angle (deg): 0.0 	 threshold ge: 315.3011243268603
Prep state: g
collecting counts
amplicude mode True
[3834. 1166.]
Prep state: e
collecting counts
amplicude mode True
[1107. 3893.]


  0%|          | 0/10 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.249
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.252
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.246
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.279
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.252
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.247
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.248
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 500
avgi 0.251
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robu

  0%|          | 0/2 [00:00<?, ?it/s]

...

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.291
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.291
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.326
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.341
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.339
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.343
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.349
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7000
avgi 0.334
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,

  0%|          | 0/2 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
Qubit  (1)
g unrotated averages:
Amps 271.7679498759331 +/- 64.03197065231396
e unrotated averages:
Amps 366.00578799190345 +/- 67.97857330110759
ge fidelity (%): 54.94000000000001 	 angle (deg): 0.0 	 threshold ge: 320.6627113034843
Prep state: g
collecting counts
amplicude mode True
[3945. 1055.]
Prep state: e
collecting counts
amplicude mode True
[1206. 3794.]


  0%|          | 0/10 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.249
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.256
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.251
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.241
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.258
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.286
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.266
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3209833086257, gain: 7500
avgi 0.274
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,

In [211]:
# fig, ax = plt.subplots(figsize=(6,4))   

# for i in range(len(p_tab)):
#     ax.plot(x_tab[i], p_tab[i], label=f'Gain {gain_scale[i]:.2f}')

# ax.set_xlabel('Number of pulses')
# ax.set_ylabel('Average intensity')
# ax.legend()

fig, ax = plt.subplots(figsize=(6,4))

ax.plot(gain_scale, p_tab, 'o-')
    

In [212]:
gain_corr =  gain*0.8
# save the new gain in the yaml file
yaml_cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(gain_corr)

In [213]:
gain_corr

4000.0

# $\pi$-$\pi$ Exp

In [214]:
qubit_i= 1


In [215]:
piminuspi.cfg.device.qubit.pulses.pihalf_ge_robust.gain

[7553,
 7553,
 7553,
 7553,
 7000,
 7500,
 4957,
 4957,
 20111,
 20111,
 20111,
 20111,
 2375,
 2375,
 2375,
 2375]

In [216]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)
reps = 100
rounds = 1
span_f = 10
npts_f = 50

# with open(config_path, 'r') as cfg_file:
#     yaml_cfg = yaml.safe_load(cfg_file)
# yaml_cfg = AttrDict(yaml_cfg)
piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))
print(piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i])



piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=1,
    expts_N=10,
    start_f=piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(reps*rounds),
    loops=2,
    pulse_type='robust',
    test_pi_half=True, 
    checkEF=False,
    qTest=qubit_i,
    qZZ=None,
    post_process='threshold',
    singleshot_reps=15000,
    full_mux_expt=True,
    resonator_reset = [0, 2, 3]
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

3448.3209833086257


  0%|          | 0/2 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
Qubit  (1)
ge fidelity (%): 55.58000000000002 	 angle (deg): 0.0 	 threshold ge: 313.87826451331205
collecting counts
amplicude mode True
collecting counts
amplicude mode True
thresholds=[0, 313.87826451331205, 0, 0],
angles=[0, 0.0, 0, 0],
ge_avgs=[array([0., 0., 0., 0.]), [99.80212134526717, 240.29590225115274, 155.29484524003254, 322.84605023053973], array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
counts_calib=[[11566.0, 3434.0], [3256.0, 11744.0]]


  0%|          | 0/2 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3443.5530242338505, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3443.7530242338503, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3443.9530242338506, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3444.1530242338504, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3444.3530242338506, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3444.5530242338505, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3444.7530242338503, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3444.9530242338506, gain: 4000
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3445.1530242338504, gain: 4000
(3, 500)
(3, 500)
(500,)
(1,

In [218]:
data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

yaml_cfg.device.qubit.f_ge_robust = np.reshape(yaml_cfg.device.qubit.f_ge_robust, (4,4))
yaml_cfg.device.qubit.f_ge_robust[qubit_i, qubit_i] = data["best_freq"]
# results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
# print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
# print(results_Qi)
print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge_robust)
yaml_cfg.device.qubit.f_ge_robust = yaml_cfg.device.qubit.f_ge_robust.flatten().tolist()
print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge_robust}')



Fit best freq 3448.3222547596297 which is 0.0012714510039586457 away from old freq 3448.3209833086257
*New f_ge
 [[4114.31381566 4108.14770152 4114.31037246 4114.31037246]
 [3450.05408931 3448.32225476 3450.05408931 3450.05408931]
 [4763.941      4763.941      4763.941      4763.941     ]
 [4384.25034226 4384.25034226 4384.25034226 4384.25034226]]
*New f_ge flattened
[4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3450.0540893078382, 3448.3222547596297, 3450.0540893078382, 3450.0540893078382, 4763.941, 4763.941, 4763.941, 4763.941, 4384.25034225973, 4384.25034225973, 4384.25034225973, 4384.25034225973]


In [ ]:
piminuspi.save_data()

# Second round of error amplification

In [227]:
qubit_i = 1
# with open(config_path, 'r') as cfg_file:
#     yaml_cfg = yaml.safe_load(cfg_file)
# yaml_cfg = AttrDict(yaml_cfg)
# gain = yaml_cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5]
# print(yaml_cfg.device.qubit.f_ge_robust[qubit_i*5])
# print(f'New gain for qubit {qubit_i} is {gain}')

In [231]:
npulsecalib = meas.NPulseExperiment(
        soccfg=soc,
        path=expt_path,
        prefix=f"NPulseExptQ{qubit_i}",
        config_file=config_path,
    )


# # 



# npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
# npulsecalib.cfg.device.qubit.pulses.pihalf_ge_robust.gain[qubit_i*5] = int(7250.396836039212)
 

npulsecalib.cfg.expt = dict(
    start=1, # number gate repetitions
    step=1,
    expts=5,
    reps=20000,
    loops=1,
    pulse_type='robust',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=True,
    error_amp=True,
    skip_first_pi2 = False, 
    post_process='threshold',
    singleshot_reps=10000,
    resonator_reset=[0, 2, 3], 
    full_mux_expt = True,
)

# print(npulsecalib.cfg)
import Pyro4.util
try:
    npulsecalib.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
Qubit  (1)
g unrotated averages:
Amps 268.5407626398797 +/- 62.713222543642985
e unrotated averages:
Amps 348.5079244918062 +/- 72.48390974764148
ge fidelity (%): 46.92000000000001 	 angle (deg): 0.0 	 threshold ge: 311.405290112306
Prep state: g
collecting counts
amplicude mode True
[7608. 2392.]
Prep state: e
collecting counts
amplicude mode True
[2934. 7066.]


  0%|          | 0/5 [00:00<?, ?it/s]

(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3222547596297, gain: 4101
avgi 0.48885
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3222547596297, gain: 4101
avgi 0.4808
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3222547596297, gain: 4101
avgi 0.51225
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3222547596297, gain: 4101
avgi 0.4509
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q1 freq_MHz: 3448.3222547596297, gain: 4101
avgi 0.52805


In [232]:
data = npulsecalib.data
npulsecalib.analyze(fit=True, scale=None)
npulsecalib.display(fit=True, scale=None)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 4101 / 0.8834126388767475 = 4642.224731145335
From avgi: adjust amplitude to 4101 / 0.9949212632477009 = 4121.934218807617
From avgq: adjust amplitude to 4101 / 1.116587162507533 = 3672.7988084605377


# Test with 1Q tomo

In [61]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=1)
import qutip as qt

meas_order = np.array(['Z', 'X', 'Y'])
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

In [77]:
tomo_analysis = TomoAnalysis(nb_qubits=1)
meas_order = np.array(['Z', 'X', 'Y'])
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)



qubit = 1 # which qubit/resonators to do tomo on
init_state = '|0>|0+i>' # switch, input
play_pulses = [0]

pi_half_swaps = None
cool_qubits = None
add_phase = False
readout_cool = False
use_IQ_pulse = False
use_robust_pulses = True # use robust pulses for state prep + tomo
ZZ_qubit = None

print('play_pulses', play_pulses)

apply_ps = False
ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psi_id = (psiZ[1] + psiZ[0]).unit()
rho_id = qt.ket2dm(psi_id)
print(rho_id)

play_pulses [0]
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]


In [78]:
# saved_files = []
qramExpt = meas.QramProtocol1QTomoExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)

span = np.inf
npts = 1
if span < np.inf: print('time step size [cycles]:', soc.us2cycles(span/npts))

qramExpt.cfg.expt = dict(
    start=span,
    step=0,
    expts=1, # number of different time experiments
    reps=50000, # number of reps per tomo/time step,
    # reps=2000000, # number of reps per tomo/time step,
    singleshot_reps=20000, # reps per state for singleshot calibration
    post_process='threshold', 
    qubit=qubit,
    init_state=init_state,
    play_pulses=play_pulses,
    cool_qubits=cool_qubits,
    use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
    use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
    plot_IQ=False,
    add_phase=add_phase,
    ZZ_qubit=ZZ_qubit,
    pi_half_swaps=pi_half_swaps,
    resonator_reset=[0, 2, 3],
    full_mux_expt = True,  
)

try:
    qramExpt.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
Y_robust_half_q1 fre

  0%|          | 0/3 [00:00<?, ?it/s]

X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q0 freq_MHz: 4114.313815659153, gain: 7553
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q1 freq_MHz: 3450.0540893078382, gain: 4957
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q2 freq_MHz: 4763.941, gain: 20111
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
X_robust_half_q3 freq_MHz: 4384.25034225973, gain: 2375
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)


KeyboardInterrupt: 

In [77]:
tomo = qramExpt


In [78]:
data = tomo.data
cfg = tomo.cfg
init_state = tomo.cfg.expt.init_state
qubit = tomo.cfg.expt.qubit
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)

print('meas_order', meas_order)

n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data=data, cfg=cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, tomo_qubits=[qubit], apply_ps=apply_ps, verbose=True)

print('Tomography counts')
for r in range(n_tomo_raw.shape[0]):
    n_tomo_raw[r,:]/=np.sum(n_tomo_raw[r,:])
print(n_tomo_raw)
print('Confusion matrix counts')
for r in range(n_calib.shape[0]):
    n_calib[r,:]/=np.sum(n_calib[r,:])
print(n_calib)
print('Confusion matrix corrected tomo matrix will be')
print(np.around(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib), decimals=5))
# print('Negative counts corrected tomo matrix will be')
# print(tomo_analysis.fix_neg_counts(tomo_analysis.correct_readout_err(n_tomo_raw, n_calib)))

meas_order ['Z' 'X' 'Y']
Tomography counts
[[0.47068 0.52932]
 [0.46352 0.53648]
 [0.70518 0.29482]]
Confusion matrix counts
[[0.73415 0.26585]
 [0.229   0.771  ]]
Confusion matrix corrected tomo matrix will be
[[0.47843 0.52157]
 [0.46426 0.53574]
 [0.94265 0.05735]]


In [79]:
# methods = 'Nelder-Mead' 'Powell' 'CG' 'BFGS' 'Newton-CG' 'L-BFGS-B' 'TNC' 'COBYLA' 'SLSQP' 'dogleg' 'trust-ncg'
rho_MLE = tomo_analysis.get_rho_from_counts(
    n_tomo_raw=n_tomo_raw,
    n_calib=n_calib,
    correct_readout=True,
    correct_neg_counts=True,
    method='analytical',
    ZZ_correction=False,
)

if rho_MLE is not None:
    fid_base = qt.fidelity(qt.Qobj(rho_MLE, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
    purity_base = np.real(np.trace(rho_MLE @ rho_MLE))
    print(f'Fidelity (no ZZ correction): {fid_base}')
    print(f'Purity (no ZZ correction): {purity_base}')
print(np.around(rho_MLE, decimals=3))
savetitle=None
# savetitle=saved_files[-1][:-3]+f'_baseMLE_Q{qubit}.svg'
tomo_analysis.show_plot_rho_2d(rho_MLE, rho_id.full(), title=f'Base MLE (Q{qubit}) Init {init_state}, Play {play_pulses}', cmax=0.5, savetitle=savetitle)

# print('Saved svg:', savetitle)
# print('Saved data:')
# print(*saved_files)

Fidelity (no ZZ correction): 0.46425814167853996
Purity (no ZZ correction): 0.8953645857845632
[[ 0.478-0.j    -0.036-0.443j]
 [-0.036+0.443j  0.522-0.j   ]]


## run the 2Q tomo

In [80]:
tomo_analysis = TomoAnalysis(nb_qubits=2)

In [95]:
def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)

    
psi_basis_3q = dict() # 27*8 elements
# psi_tensor = [psiZ[0], psiZ[0], psiZ[0], psiZ[0]]
S123 = 'ZZZ'
S1, S2, S3 = S123
psi1 = psi_dict[S1]
psi2 = psi_dict[S2]
psi3 = psi_dict[S3]

psi_basis_3q.update({S123:[]})

for state1 in (0,1):
    for state2 in (0,1):
        for state3 in (0,1):
            psi_basis_3q[S123].append(qt.tensor(psi1[state1], psi2[state2], psi3[state3]).unit())
calib_order_numeric_3q = ['000', '001', '010', '011', '100', '101', '110', '111']
psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(calib_order_numeric_3q):
    psis.update({label_numeric:psi_basis_3q['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))

def get_qram_qSLR_state_from_rho(init_rho_IS, output_qubits, post_select=False, post_select_state=None):
    """
    init_rho_IS should be a np array
    outputs partial trace of qram output (needs output_qubits to all be in 0, 2, and/or 3 to work)
    """
    # in order |00>, |01>, |10>, |11> for switch, input
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]

    assert np.all([q in [0, 2, 3] for q in output_qubits])
    assert np.shape(init_rho_IS) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    print(f'constructing final state on |Q{output_qubits[0]}, Q{output_qubits[1]}>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_IS[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    output_indices = [[0,2,3].index(q) for q in output_qubits]
    return qt.ptrace(rho_final_SLR.unit(), output_indices)

In [106]:
tomo_qubits = [0, 1] # which qubit/resonators to do tomo on

init_state = '|1>|0+i>' # Q0/Q1 (switch, input)
# init_state = 'Q3Q0_|1>|1>'
# init_state = '|1>|0>' # Q0/Q1 (switch, input)
# init_state = 'Q2Q0_|1>|0>'
# init_state = '|1>|1>' # Q0/Q1 (switch, input)
# play_pulses = [2, 1, 4, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2, 4] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [1, 3] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
play_pulses = [0] # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]
# play_pulses = [2]*8 # 1: Q2/Q1 with Q0 in g, 2: Q3/Q1 with Q0 in e, 3: Q2 ef pulse, 4: Q3 ef pulse [2, 1, 4, 3]

# cool_qubits = [0, 1]
cool_qubits = None


readout_cool = False

use_IQ_pulse = False # state transfer IQ pulses for the state prep
use_robust_pulses = True # use robust pulses for state prep + tomo

post_select = False # this is not the readout post selection

# add_phase = True # phase for swaps
add_phase = False
print('init_state', init_state)
print('post select', post_select)
ps_adjust = [-0.3, -1.0, -0.3, -0.3]
# ps_adjust = [np.inf]*4
ps_qubits = [0, 1, 2, 3]
use_ZZ_mat = True

rho_id = qt.ket2dm(name_to_state_2q(init_state))

evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"
print('Will save evol mats to path', evol_mats_path)

q0, q1 = tomo_qubits
evol_mats_filename = f'evol_mats_{q0}{q1}.npz'


init_state |1>|0+i>
post select False
Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats


In [107]:
qramExpt = meas.QramProtocolExperiment(
        soccfg=soc,
        path=expt_path,
        config_file=config_path,)


qramExpt.cfg.expt = dict(
        # start=0, # start protocol time [us],
        # step=span/npts, # time step [us], 
        # expts=npts+1, # number of different time experiments - added 1 to include 0 step and full span

        start=np.inf,
        step=0,
        expts=1, # number of different time experiments
        reps=10000, # number of reps per time step,
        # singleshot_reps=100, # reps per state for singleshot calibration
        singleshot_reps=10000, # reps per state for singleshot calibration
        post_process='threshold', # 'threshold', (uses single shot binning), 'scale' (scale by ge_avgs), or None
        tomo_2q=True, # True/False whether to do 2q state tomography on state at last time step
        tomo_qubits=tomo_qubits, # the qubits on which to do the 2q state tomo
        calib_apply_q1_pi2=False, # initialize Q1 to 0+1 for all calibrations
        init_state=init_state,
        play_pulses=play_pulses,
        cool_qubits=cool_qubits,
        
        use_IQ_pulse=use_IQ_pulse, # state transfer IQ pulses for the state prep
        use_robust_pulses=use_robust_pulses, # use robust pulses for state prep + tomo
        plot_IQ=False,
        full_mux_expt = True,
        resonator_reset=[0, 2, 3],
        
        add_phase=add_phase,
        
        # readout_cool=readout_cool,
        # n_init_readout=1,
        # n_trig=1,
        # # init_read_wait_us=5,
        # init_read_wait_us=10,
        
        # Icontrols=I_values_MHz, # 2D array with array of Icontrols for each of IQ_qubits
        # Qcontrols=Q_values_MHz, # 2D array with array of Qcontrols for each of IQ_qubits
        # times_us=times_us,
        # IQ_qubits=IQ_qubits,
        # IQ_gain=[3700, 3950],
        
        # thresholds=[33.169103683345895, 37.49913270024612, 0, 0],
        # angles=[-24.579056838557847, 115.19639444481155, 0, 0],
        # ge_avgs=[[34.43056101813111, -9.525004532775455, 44.589988819153874, -4.651175313807532], [30.053312459336368, -50.17723301886793, 25.548312166558233, -59.65660634352635], array([0., 0., 0., 0.]), array([0., 0., 0., 0.])],
        # counts_calib=[[12553, 5207, 1604, 636], [4673, 12956, 666, 1705], [2241, 876, 12383, 4500], [918, 2095, 4712, 12275]]
 
    )

    # print(qramExpt.cfg)

import Pyro4.util
try:
    qramExpt.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

timesteps [inf]


  0%|          | 0/4 [00:00<?, ?it/s]

using X_pulse
qubit 0
special None
qubit 0
special robust
qubit 0
special robust
ENFORCING GAUSSIAN PULSE for qubit 1
using X_pulse
qubit 1
special gauss
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
using X_pulse
qubit 2
special None
qubit 2
special robust
qubit 2
special robust
using X_pulse
qubit 3
special None
qubit 3
special robust
qubit 3
special robust
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
ADC readout length 1229
ADC readout length 1229
ADC readout length 1229
ADC readout length 1229
using X_pulse
qubit 0
special None
qubit 0
special robust
qubit 0
special robust
ENFORCING GAUSSIAN PULSE for qubit 1
using X_pulse
qubit 1
special gauss
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
using X_pulse
qubit 2
special None
qubit 2
special robust
qubit 2
special robust
using X_pulse
qubit

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

using X_pulse
qubit 0
special None
qubit 0
special robust
qubit 0
special robust
ENFORCING GAUSSIAN PULSE for qubit 1
using X_pulse
qubit 1
special gauss
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
using X_pulse
qubit 2
special None
qubit 2
special robust
qubit 2
special robust
using X_pulse
qubit 3
special None
qubit 3
special robust
qubit 3
special robust
(3, 500)
(3, 500)
(500,)
(1, 500)
(1, 500)
(500,)
ADC readout length 1229
ADC readout length 1229
ADC readout length 1229
ADC readout length 1229
using X_pulse
qubit 0
special gauss
qubit 0
special gauss
using gauss pulse here
qubit 0
using gauss pulse
qubit 0
special gauss
using gauss pulse here
qubit 0
using gauss pulse
ENFORCING GAUSSIAN PULSE for qubit 1
using X_pulse
qubit 1
special gauss
qubit 1
special gauss
using gauss pulse here
qubit 1
using gauss pulse
ZZ [ 614  565 3727 5094]
using X_pulse
qubit 0
special None
qubit 0
specia

In [186]:
tomo = qramExpt
data = tomo.data
cfg = tomo.cfg
init_state = tomo.cfg.expt.init_state
# qubit = tomo.cfg.expt.qubit
pulse_dict = tomo.pulse_dict
assert np.all(np.array(tomo.meas_order) == tomo_analysis.meas_order)

In [109]:
# run_evol = True 

# if run_evol:
#     evol_mats = tomo_analysis.get_evol_mats_from_yaml(
#         tomo_qubits=tomo_qubits,
#         yaml_cfg=yaml_cfg,
#         pulse_dict=pulse_dict,
#         cutoffs=[4]*(len(tomo_qubits)),
#         soccfg=soc,
#         debug=True,
#         evol_mats_path=evol_mats_path,
#         evol_mats_filename=evol_mats_filename,
#     )
    

    
# else:
#     evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
#     print(f'Using evol mats from file {evol_mats_file_path}')

#     evol_mats = dict()
#     with np.load(evol_mats_file_path) as npzfile:
#         for key in npzfile.keys():
#             evol_mats.update({key:npzfile[key]})

In [110]:
n_tomo_raw, n_calib = tomo_analysis.n_tomo_calib_from_data(data, cfg, ps_adjust=ps_adjust, ps_qubits=ps_qubits, apply_ps=apply_ps, verbose=True)
    
rho_MLE_ZZ = tomo_analysis.get_rho_from_counts(
        n_tomo_raw=n_tomo_raw,
        n_calib=n_calib,
        correct_readout=True,
        correct_neg_counts=True,
        method='analytical',
        ZZ_correction=True,
        evol_mats=evol_mats
    )

print(np.around(rho_MLE_ZZ, decimals=10))
print('list format', np.around(rho_MLE_ZZ, decimals=7).tolist())
fid_ZZ = qt.fidelity(qt.Qobj(rho_MLE_ZZ, dims=rho_id.dims), rho_id)**2 # qutip uses N&C fidelity which is "sqrt fidelity"
purity_ZZ = np.real(np.trace(rho_MLE_ZZ @ rho_MLE_ZZ))
purity_id = np.real(np.trace(rho_id.full() @ rho_id.full()))
print(f'Fidelity (with ZZ correction): {fid_ZZ}')
print(f'Purity (with ZZ correction): {purity_ZZ}')
print(f'Target purity: {purity_id}')

tomo_analysis.show_plot_rho_2d(rho_MLE_ZZ, rho_id.full(), title=f'ZZ MLE (Q{q0}Q{q1}) Init {init_state}, Play {play_pulses}', cmax=0.5, size=(6,6))

 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1224ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Seb\experiements\qram_tprocv1_expts\TomoAnalysis.py: 1229

[[ 0.02637547+0.j         -0.01093064-0.0027378j  -0.05267488-0.02484796j
  -0.00250557+0.0935347j ]
 [-0.01093064+0.0027378j   0.04425777+0.j         -0.08763359-0.01901523j
   0.01572976+0.03007496j]
 [-0.05267488+0.02484796j -0.08763359+0.01901523j  0.46128718+0.j
  -0.19419721-0.37068741j]
 [-0.00250557-0.0935347j   0.01572976-0.03007496j -0.19419721+0.37068741j
   0.46807958+0.j        ]]
list format [[(0.0263755+0j), (-0.0109306-0.0027378j), (-0.0526749-0.024848j), (-0.0025056+0.0935347j)], [(-0.0109306+0.0027378j), (0.0442578+0j), (-0.0876336-0.0190152j), (0.0157298+0.030075j)], [(-0.0526749+0.024848j), (-0.0876336+0.0190152j), (0.4612872+0j), (-0.1941972-0.3706874j)], [(-0.0025056-0.0935347j), (0.0157298-0.030075j), (-0.1941972+0.3706874j), (0.4680796+0j)]]
Fidelity (with ZZ correction): 0.8353708040932414
Purity (with ZZ correction): 0.8277166205516706
Target purity: 0.9999999999999996


In [ ]:
psi = qt.basis(3,0)

In [ ]:
rho = psi*psi.dag()

In [ ]:
rho.extract_states([0])